In [1]:
# In[23]:

import quandl
quandl.get('FRED/GDP').plot(kind='line')


# In[138]:

import pandas as pd
import numpy as np
from collections import OrderedDict

import quandl
quandl.get('FRED/GDP').plot(kind='line')

class income_statement:
    def __init__(self, name, sales, cogs, sga, da, interest, tax_rate):
        self.name = name
        self.sales = sales
        self.cogs = cogs
        self.gross_profit = sales-cogs
        self.sga = sga
        self.ebitda = self.gross_profit - sga
        self.da = da
        self.ebit = self.ebitda - da
        self.interest = interest
        self.ebt = self.ebit-interest
        self.tax = tax_rate*self.ebt
        self.net_income = self.ebt - self.tax
        self.nopat = (1-tax_rate)*self.ebit
    def get_income_statement(self, depreciation_rate=0, delta_capex=0, delta_workingcap=0):
        dat = {'Sales':self.sales, 'COGS':self.cogs, 'GrossProfit':self.gross_profit,
              'SG&A':self.sga, 'EBITDA':self.ebitda, 'Depreciation':self.da,
              'EBIT':self.ebit, 'Interest':self.interest, 'EBT':self.ebt, 
              'Taxes':self.tax, 'NetIncome':self.net_income, 'NOPAT':self.nopat,
              'NetCapEx':delta_capex, 'NetWorkingCap':delta_workingcap,
              'FreeCashFlow':self.nopat+depreciation_rate-delta_capex-delta_workingcap} 
        deezcolz = ['Sales','COGS','GrossProfit','SG&A','EBITDA','Depreciation','EBIT',
                   'Interest','EBT','Taxes','NetIncome','NOPAT','NetCapEx','NetWorkingCap','FreeCashFlow']
        df = pd.DataFrame(dat, index=[self.name], columns=deezcolz)#.transpose()
        return df.transpose()
    def print_statement(self):
        print('''
        Income Statement
        _________________________________________________________________________
        
        Sales/Revenue ......................................... %.2f
        -Cost of Goods Sold ......................................... %.2f
        _________________________________________________________________________
        Gross Profit ......................................... %.2f
        -Sales General & Admin. ......................................... %.2f
        _________________________________________________________________________
        EBITDA ......................................... %.2f
        -Depreciation/Amortization ......................................... %.2f
        _________________________________________________________________________
        EBIT ......................................... %.2f
        -Interest Expense ......................................... %.2f
        _________________________________________________________________________
        EBT ......................................... %.2f
        -Taxes ......................................... %.2f
        _________________________________________________________________________
        Net Income ......................................... %.2f
        
        '''%(self.sales, self.cogs, self.gross_profit, self.sga, self.ebitda, self.da, self.ebit, 
            self.interest, self.ebt, self.tax, self.net_income))


# In[2]:

class balance_sheet:
    def __init__(self, cash, ar, inventory, ppe, ap, longterm_debt, stock, retained):
        ## Assets
        self.cash = cash
        self.ar = ar
        self.inventory = inventory
        self.total_current_assets = cash+ar+inventory
        self.ppe = ppe
        self.total_longterm_assets = ppe +0#placeholder for extensions
        self.total_assets = self.total_current_assets + self.total_longterm_assets
        ## Liabilities & equity
        self.ap = ap
        self.total_current_liabilities = ap +0#placeholder for extensions
        self.longterm_debt = longterm_debt
        self.total_liabilities = self.total_current_liabilities + self.longterm_debt
        self.stock = stock
        self.retained = retained
        self.total_equity = stock + retained
        self.total_liabilities_equity = self.total_liabilities + self.total_equity
    def get_balance_sheet(self):
        pass
    def print_balance_sheet(self):
        print('''
        Assets
        ___________________________________________________________
        Cash & Equivalents .............................. %.4f
        Accounts Receivable .............................. %.4f
        Inventories .............................. %.4f
        ___________________________________________________________
        Total Current Assets .............................. %.4f
        ___________________________________________________________
        Net Property Plant & Equipment .............................. %.4f
        Total Long-Term Assets .............................. %.4f
        ___________________________________________________________
        Total Assets .............................. %.4f
        
        Liabilities
        ___________________________________________________________
        Accounts Payable .............................. %.4f
        ___________________________________________________________
        Total Current Liabilities .............................. %.4f        
        ___________________________________________________________
        Long-Term Debt .............................. %.4f
        ___________________________________________________________
        Total Liabilities .............................. %.4f
        
        Equity
        ___________________________________________________________
        Common Stock .............................. %.4f
        Retained Earnings .............................. %.4f
        ___________________________________________________________
        Total Stockholders Equity .............................. %.4f
        ___________________________________________________________
        Total Liabilities & Equity .............................. %.4f
        
        '''%(self.cash, self.ar, self.inventory, self.total_current_assets, self.ppe, self.total_longterm_assets,
             self.total_assets, self.ap, self.total_current_liabilities, self.longterm_debt, self.total_liabilities,
             self.stock, self.retained, self.total_equity, self.total_liabilities_equity) )



In [2]:
class financial_metrics:
    def __init__(self, income_statement, balance_sheet):
        ## Income statement
        self.sales = income_statement.sales
        self.cogs = income_statement.cogs
        self.gross_profit = income_statement.gross_profit
        self.sga = income_statement.sga
        self.ebitda = income_statement.ebitda
        self.da = income_statement.da
        self.ebit = income_statement.ebit
        self.interest = income_statement.interest
        self.ebt = income_statement.ebt
        self.tax = income_statement.tax
        self.net_income = income_statement.net_income
        self.nopat = income_statement.nopat
        self.gross_margin = self.gross_margin()
        self.operating_margin = self.operating_margin()
        self.net_margin = self.net_margin()
        ## Balance sheet
        ## Assets
        self.cash = balance_sheet.cash
        self.ar = balance_sheet.ar
        self.inventory = balance_sheet.inventory
        self.total_current_assets = balance_sheet.total_current_assets
        self.ppe = balance_sheet.ppe
        self.total_longterm_assets = balance_sheet.total_longterm_assets
        self.total_assets = balance_sheet.total_assets
        ## Liabilities
        self.ap = balance_sheet.ap
        self.total_current_liabilities = balance_sheet.total_current_liabilities
        self.longterm_debt = balance_sheet.longterm_debt
        self.total_liabilities = balance_sheet.total_liabilities
        self.stock = balance_sheet.stock
        self.retained = balance_sheet.retained
        self.total_equity = balance_sheet.total_equity
        self.total_liabilities_equity = balance_sheet.total_liabilities_equity
        ## Metrics
        self.asset_turnover = self.asset_turnover()
        self.inventory_turnover = self.inventory_turnover()
        self.ar_turnover = self.ar_turnover()
        self.ap_turnover = self.ap_turnover()
        self.roa = self.return_on_assets()
        self.roe = self.return_on_equity()
        self.invested_capital = self.invested_capital()
        self.roic = self.return_on_invested_capital()
        self.gmroi = self.gmroi()
        
    def gross_margin(self):
        return self.gross_profit / self.sales
    def operating_margin(self):
        return self.ebitda / self.sales
    def net_margin(self):
        return self.net_income / self.sales
    def asset_turnover(self):
        return self.sales / self.total_assets
    def inventory_turnover(self):
        return self.cogs / self.inventory
    def ar_turnover(self):
        return self.sales / self.ar
    def ap_turnover(self):
        return self.cogs / self.ap
    def return_on_assets(self):
        return self.net_income / self.total_assets
    def return_on_equity(self):
        return self.net_income / self.total_equity
    def invested_capital(self):
        return self.total_equity + self.longterm_debt
    def return_on_invested_capital(self):
        return self.nopat / self.invested_capital
    def gmroi(self):
        return self.gross_margin * self.inventory_turnover
    def print_metrics(self):
        print('''
        Financial Metrics:
        _______________________________________________________________
        Gross Margin .............................. %.5f
        Operating Margin .............................. %.5f
        Net Margin .............................. %.5f
        Asset Turnover .............................. %.5f
        Inventory Turnover .............................. %.5f
        AR Turnover .............................. %.5f
        AP Turnover .............................. %.5f
        Return on Assets .............................. %.5f
        Return on Equity .............................. %.5f
        NOPAT .............................. %.5f
        Invested Capital .............................. %.5f
        Return on Invested Capital .............................. %.5f
        Gross Margin Return on Inventory .............................. %.5f
        
        '''%(self.gross_margin, self.operating_margin, self.net_margin, self.asset_turnover, self.inventory_turnover,
            self.ar_turnover, self.ap_turnover, self.roa, self.roe, self.nopat, self.invested_capital, 
            self.roic, self.gmroi))

income_b = income_statement(name='bonsa', sales=14000, cogs=11000, sga=1200, da=0, interest=240, tax_rate=.4)
income_b.print_statement()
        
balance_b = balance_sheet(cash=500, ar=3000, inventory=2800, ppe=5000, ap=900, longterm_debt=4800, stock=4200, retained=1400)
balance_b.print_balance_sheet()

metrics_b = financial_metrics(income_b, balance_b)
metrics_b.print_metrics()


        Income Statement
        _________________________________________________________________________
        
        Sales/Revenue ......................................... 14000.00
        -Cost of Goods Sold ......................................... 11000.00
        _________________________________________________________________________
        Gross Profit ......................................... 3000.00
        -Sales General & Admin. ......................................... 1200.00
        _________________________________________________________________________
        EBITDA ......................................... 1800.00
        -Depreciation/Amortization ......................................... 0.00
        _________________________________________________________________________
        EBIT ......................................... 1800.00
        -Interest Expense ......................................... 240.00
        _______________________________________

In [3]:
income_o = income_statement(name='osun', sales=5000, cogs=3500, sga=600, da=0, interest=0, tax_rate=.3)
income_o.print_statement()
        
balance_o = balance_sheet(cash=500, ar=550, inventory=700, ppe=2500, ap=550, longterm_debt=0, stock=2900, retained=800)
balance_o.print_balance_sheet()

metrics_o = financial_metrics(income_o, balance_o)
metrics_o.print_metrics()


        Income Statement
        _________________________________________________________________________
        
        Sales/Revenue ......................................... 5000.00
        -Cost of Goods Sold ......................................... 3500.00
        _________________________________________________________________________
        Gross Profit ......................................... 1500.00
        -Sales General & Admin. ......................................... 600.00
        _________________________________________________________________________
        EBITDA ......................................... 900.00
        -Depreciation/Amortization ......................................... 0.00
        _________________________________________________________________________
        EBIT ......................................... 900.00
        -Interest Expense ......................................... 0.00
        ______________________________________________

In [4]:
## GA1 W10 Osun 
## Osun FY0 not FY1 (above)
income_o = income_statement(name='osun', sales=6000, cogs=3900, sga=700, da=0, interest=0, tax_rate=.3)
income_o.print_statement()
        
balance_o = balance_sheet(cash=500, ar=700, inventory=1000, ppe=3500, ap=600, longterm_debt=0, stock=4100, retained=1000)
balance_o.print_balance_sheet()

metrics_o = financial_metrics(income_o, balance_o)
metrics_o.print_metrics()


        Income Statement
        _________________________________________________________________________
        
        Sales/Revenue ......................................... 6000.00
        -Cost of Goods Sold ......................................... 3900.00
        _________________________________________________________________________
        Gross Profit ......................................... 2100.00
        -Sales General & Admin. ......................................... 700.00
        _________________________________________________________________________
        EBITDA ......................................... 1400.00
        -Depreciation/Amortization ......................................... 0.00
        _________________________________________________________________________
        EBIT ......................................... 1400.00
        -Interest Expense ......................................... 0.00
        ____________________________________________

In [159]:
def days_payable_outstanding(AP, COGS):
    '''Avg AP & COGS'''
    return (AP/COGS)*365

def days_inventory_outstanding(avg_inventory, COGS):
    '''Avg Inventory & COGS'''
    return (avg_inventory/COGS)*365

def days_sales_outstanding(AR, sales):
    '''Avg AR & COGS'''
    return (AR/sales)*365

def cash_conversion_cycle(DIO, DSO, DPO):
    '''Number of days cash tied up in Biz'''
    return DIO + DSO - DPO

def depreciation(buy, horizon, salvage_period, salvage=0, method='straight'):
    depreciable_amt = (buy-salvage)/horizon
    df = pd.DataFrame({'Period':np.arange(0, horizon+1),
                      'Depreciation':depreciable_amt})
    df.set_index('Period', inplace=True)
    df.loc[0, 'Value'] = buy
    for i, row in df.iterrows():
        if i == 0:
            pass
        else:
            df.loc[i, 'Value'] = df.loc[i-1, 'Value'] - row['Depreciation']
    df.loc[0:salvage_period, 'Own'] = True
    liquidate_value = df.loc[salvage_period, 'Value']
    df.Own.fillna(False, inplace=True)
    df = df.loc[df.Own==True]
    df.drop(labels='Own', axis=1, inplace=True)
    #df = df.transpose()
    print('''
    
    Liquidation Value ............................ %.2f
    Yearly Depreciation ............................ %.2f
    
    ''' %(liquidate_value, depreciable_amt))
    print(df.transpose())
    return liquidate_value

def ebit(sales, cogs, opex, da):
    return sales - cogs - opex - da

def nopat(tax_rate, ebit):
    return (1-tax_rate)*ebit

def free_cash_flow(nopat, depreciation, capex, delta_working_capital):
    fcf = nopat + depreciation - capex - delta_working_capital
    return fcf

def delta_working_capital(net_AR, net_inventory, net_AP):
    return net_AR + net_inventory - net_AP

def payback_period(cashflows):
    CF0 = abs(cashflows[0])
    CF_future = cashflows[1:]
    df = pd.DataFrame({'Inflows':CF_future, 
                 'Original':CF0},
                index=np.arange(1,len(cashflows)))
    df['CumInflow'] = df.Inflows.cumsum()
    df['Left'] = df.Original - df.CumInflow
    df.index.name = 'Period'
    df = df.loc[df.Left < 0]
    payback_period = round(df.index.values-1 + (1 - ((df.CumInflow - df.Original) / df.Inflows)),3)
    payback_period = np.float64(np.min(payback_period))
    return payback_period

def delta_balsheet_item(item_list):
    ls = []
    for i, item in enumerate(item_list):
        if i == 0:
            ls.append(item)
        elif i < len(item_list)+1:
            ls.append(item-item_list[i-1])
        else:
            pass
    return ls

def perpetuity(CF, r):
    return CF/r

In [73]:
## W8 PP5 Cash to Cash Cycle

df = pd.DataFrame({'Company':['UNI','P&G','KC'],
                  'AvgInventory':[4053,6834,2063],
                  'AvgAR':[4930,6447,2384],
                  'AvgAP':[12171,8619,2607], 
                  'COGS':[40446,42460,13041],
                  'Sales':[48436,83062,19724]})

for i, row in df.iterrows():
    DIO = days_inventory_outstanding(row['AvgInventory'], row['COGS'])
    DSO = days_sales_outstanding(row['AvgAR'], row['Sales'])
    DPO = days_payable_outstanding(row['AvgAP'], row['COGS'])
    CO = row['Company']
    CCC = cash_conversion_cycle(DIO, DSO, DPO)
    print('''
    Company .......................... %s
    Days Inventory Outstanding .......................... %.2f
    Days Sales Outstanding .......................... %.2f
    Days Payables Outstanding .......................... %.2f
    Cash Conversion Cycle .......................... %.2f
    ''' %(CO, DIO, DSO, DPO, CCC))


# In[166]:

## W9 PP1



equipment_buy = 2000
equipment_sell = -1*depreciation(buy=equipment_buy, horizon=10, salvage_period=3)
labor_savings = -450
forklift_savings = -350
floor_salaries = 300
tax_rate = 0.3
discount_rate = 0.1
depreciation_rate = 200
change_opex = labor_savings+forklift_savings+floor_salaries

## Gather income statements 
income_0 = income_statement(name='Year 0', sales=0, cogs=0, sga=0, da=0, interest=0, tax_rate=tax_rate).get_income_statement(
    depreciation_rate=0, delta_capex=equipment_buy, delta_workingcap=0)
income_1 = income_statement(name='Year 1', sales=0, cogs=0, sga=change_opex, da=depreciation_rate, interest=0, tax_rate=tax_rate).get_income_statement(
    depreciation_rate=depreciation_rate, delta_capex=0, delta_workingcap=0)
income_2 = income_statement(name='Year 2', sales=0, cogs=0, sga=change_opex, da=depreciation_rate, interest=0, 
                            tax_rate=tax_rate).get_income_statement(depreciation_rate=depreciation_rate, delta_capex=0, delta_workingcap=0)
income_3 = income_statement(name='Year 3', sales=0, cogs=0, sga=change_opex, da=depreciation_rate, interest=0, 
                            tax_rate=tax_rate).get_income_statement(depreciation_rate=depreciation_rate, delta_capex=equipment_sell, delta_workingcap=0)
income = pd.concat([income_0, income_1, income_2, income_3], axis=1)
print(income)

cashflows = income.ix['FreeCashFlow'].tolist()


PBP = payback_period(cashflows)    
NPV = np.npv(discount_rate, income.ix['FreeCashFlow'])
IRR = np.irr(income.ix['FreeCashFlow'])

print('''
NPV ................................ %.4f
IRR ................................ %.4f
Payback Period ................................ %.4f
'''%(NPV, IRR, PBP))


# In[167]:

print('''
These are the free cash flows used in IRR & NPV.
''')
print(income.ix['FreeCashFlow'][1:])


    Company .......................... UNI
    Days Inventory Outstanding .......................... 36.58
    Days Sales Outstanding .......................... 37.15
    Days Payables Outstanding .......................... 109.84
    Cash Conversion Cycle .......................... -36.11
    

    Company .......................... P&G
    Days Inventory Outstanding .......................... 58.75
    Days Sales Outstanding .......................... 28.33
    Days Payables Outstanding .......................... 74.09
    Cash Conversion Cycle .......................... 12.99
    

    Company .......................... KC
    Days Inventory Outstanding .......................... 57.74
    Days Sales Outstanding .......................... 44.12
    Days Payables Outstanding .......................... 72.97
    Cash Conversion Cycle .......................... 28.89
    

    
    Liquidation Value ............................ 1400.00
    Yearly Depreciation .........................

In [123]:
## W9 PP2 Incremental w/ perpetuity
discount_rate = .10
sls = [0,1000,2000,4000,4000]
cogs = [0,600,1200,2400,2400]
opsga = [n + 100 for n in [-100,100,200,300,300]] #add in marketing expense
ar = delta_balsheet_item([0,200,350,480,480])
inv = delta_balsheet_item([500,700,1100,2100,2100])
ap = delta_balsheet_item([200,300,550,980,980])

N = 5 #N+1
yr = ['Year '+str(i) for i in np.arange(0,N)]
df = pd.DataFrame()
for i in np.arange(0,N):
    net_AR=ar[i], 
    net_inventory=inv[i]
    net_AP=ap[i]
    
    net_wc = delta_working_capital(net_AR=ar[i], 
                                   net_inventory=inv[i], 
                                   net_AP=ap[i])
    
    inc_stm = income_statement(name=yr[i], 
                               sales=sls[i], 
                               cogs=cogs[i], 
                               sga=opsga[i], 
                               da=0, interest=0, tax_rate=.4)
    inc_stm = inc_stm.get_income_statement(depreciation_rate=0, 
                                 delta_capex=0, 
                                 delta_workingcap=net_wc).transpose()
    df = df.append(inc_stm)

df = df.transpose()

PERP_CF = df.ix['FreeCashFlow'][-1:]
df.ix['FreeCashFlow'][-2] = df.ix['FreeCashFlow'][-2] + perpetuity(PERP_CF, r=discount_rate)
cashflows = df.ix['FreeCashFlow'][:-1].tolist()


print('''
Adjusted DF with second to last period adjusted for perpetuity.
''')
df.drop(labels=df.columns[-1], axis=1, inplace=True)
print(df)


NPV = np.npv(discount_rate, cashflows)

print(PBP, NPV, IRR)

print('''
NPV w/ Perpetuity ................................ %.4f
'''%(NPV))


Adjusted DF with second to last period adjusted for perpetuity.

               Year 0  Year 1  Year 2  Year 3
Sales             0.0  1000.0  2000.0  4000.0
COGS              0.0   600.0  1200.0  2400.0
GrossProfit       0.0   400.0   800.0  1600.0
SG&A              0.0   200.0   300.0   400.0
EBITDA            0.0   200.0   500.0  1200.0
Depreciation      0.0     0.0     0.0     0.0
EBIT              0.0   200.0   500.0  1200.0
Interest          0.0     0.0     0.0     0.0
EBT               0.0   200.0   500.0  1200.0
Taxes             0.0    80.0   200.0   480.0
NetIncome         0.0   120.0   300.0   720.0
NOPAT             0.0   120.0   300.0   720.0
NetCapEx          0.0     0.0     0.0     0.0
NetWorkingCap   300.0   300.0   300.0   700.0
FreeCashFlow   -300.0  -180.0     0.0  7220.0
Series([], dtype: float64) 4960.85649887 1.7003849214487565

NPV w/ Perpetuity ................................ 4960.8565



In [166]:
## W9 PP3 comparing projects using figures of merit
r = 0.10
cashflow_df = pd.DataFrame(index=['Year '+str(i) for i in np.arange(0,4)],
                          data={'A':[-3400,0,0,6500],
                               'B':[-3400,1900,1900,1900],
                               'C':[-3400,1000,1700,3400]})

cashflow_df = cashflow_df.transpose()
cashflow_df.index.name = 'Project'
cashflow_df

def compare_merit(cashflow_df, r):
    df = pd.DataFrame(index=cashflow_df.index.values)
    df.index.name = 'Project'
    for i, row in cashflow_df.iterrows():
        df.loc[i, 'NPV@'+str(r)] = round(np.npv(r, row), 4)
        df.loc[i, 'IRR'] = round(np.irr(row), 4)
        df.loc[i, 'PBP'] = round(payback_period(row.tolist()), 4)
    return df

compare_merit(cashflow_df, r=.15)
#[payback_period(CF_stream.tolist()) for i,CF_stream in cashflow_df.iterrows()]

,NPV@0.15,IRR,PBP
Project,,,
A,873.8555,0.2411,2.523
B,938.1277,0.3106,1.789
C,990.5646,0.2867,2.206


In [237]:
## W9 PP4 Comparing financing options
r = 0.05
t = 2
periods_per_yr = 4
t_qtrs = t*periods_per_yr
down_pmt = 15000
cost_of_asset = 150000
loan_amt = cost_of_asset - down_pmt

def periodic_rate(r, periods_per_year):
    periodic_interest_rate = round(((1+r)**(1/periods_per_year))-1, 4)
    return periodic_interest_rate

def periodic_payment(periodic_r, PV_loan, periods):
    pmt = round((periodic_r * PV_loan) / (1- (1/(1+periodic_r)**periods)), 4)
    print('''
    Periodic interest rate ........................... %.3f percent
    Periodic payment ........................... %.2f
    Periods ........................... %i
    '''%(periodic_r*100, pmt, periods))
    return pmt

def generate_pmt_schedule(r, periods, pmt, down_pmt, cost_of_asset):
    print('Assumes down payment already backed out of loan.')
    t, pmt = np.arange(0,periods+1), np.repeat(pmt, periods+1)
    df = pd.DataFrame(index=t, data={'Payment':pmt,
                                    'PrincipalReduction':np.nan,
                                    'InterestPaid':np.nan})
    df.index.name = 'Period'
    df.loc[0, 'PrincipalRemaining'] = cost_of_asset - down_pmt
    df.loc[0, 'Payment'] = down_pmt
    df.loc[0, ['PrincipalReduction','InterestPaid']] = 0
    
    for i, row in df.iterrows():
        if i == 0:
            pass
        elif i == len(df.index):
            pass
        else:
            df.loc[i, 'InterestPaid'] = df.loc[i-1, 'PrincipalRemaining'].astype(np.float64)*r
            df.loc[i, 'PrincipalReduction'] = df.loc[i, 'Payment']  - df.loc[i, 'InterestPaid'] 
            df.loc[i, 'PrincipalRemaining'] = df.loc[i-1, 'PrincipalRemaining'] - df.loc[i, 'PrincipalReduction']
    round_colz = ['InterestPaid','Payment','PrincipalReduction','PrincipalRemaining']
    df[round_colz] = df[round_colz].apply(lambda x: round(x, 2))
    
    notcum_colz = ['InterestPaid','Payment']
    cum_colz = ['Cumulative'+txt for txt in notcum_colz]
    df[cum_colz] = df[notcum_colz].cumsum()
    return df

qtrly_rate = periodic_rate(r, periods_per_yr)
qtrly_pmt = periodic_payment(qtrly_rate, loan_amt, t_qtrs)

pmt_schedule = generate_pmt_schedule(r=qtrly_rate, 
                      periods=t_qtrs, 
                      pmt=qtrly_pmt, 
                      down_pmt=down_pmt, 
                      cost_of_asset=cost_of_asset)
print(pmt_schedule)

pv_at4percent = round(np.npv(.04, pmt_schedule.Payment.tolist()),2)
pv_at4percent


    Periodic interest rate ........................... 1.230 percent
    Periodic payment ........................... 17822.35
    Periods ........................... 8
    
Assumes down payment already backed out of loan.
        InterestPaid   Payment  PrincipalReduction  PrincipalRemaining  \
Period                                                                   
0               0.00  15000.00                0.00           135000.00   
1            1660.50  17822.35            16161.85           118838.15   
2            1461.71  17822.35            16360.64           102477.51   
3            1260.47  17822.35            16561.88            85915.63   
4            1056.76  17822.35            16765.59            69150.04   
5             850.55  17822.35            16971.81            52178.24   
6             641.79  17822.35            17180.56            34997.68   
7             430.47  17822.35            17391.88            17605.80   
8             216.55  17822.35      

134993.34

In [281]:
# W9 PP5 More Discounting Practice
def growing_perpetuity(C, r, g):
    return round(C / (r-g),2)

r = .08
t = [2017,2018,2019,2020,2021,2022]
g_short = .05
g_long = .03
sls = [round(1000*(1+g_short)**t,2) for t in np.arange(0,4)]
base_perpetuity = sls[-1]
sls.append(base_perpetuity*(1+g_long)**1)
sls.append(base_perpetuity*(1+g_long)**2)
sls[-1] = growing_perpetuity(C=sls[-1], r=r, g=g_long)

df = pd.DataFrame(index=t, data={'Sales':sls})
df.index.name = 'Year'
df.loc[2021, 'Sales'] = df.loc[2021, 'Sales'] + df.loc[2022, 'Sales']
df.loc[2017, 'Sales'] = 0
df.drop(labels=[2022], axis=0, inplace=True)
print(df)

np.npv(r, df.Sales.tolist())

# def incremental_freecashflows(sls, cogs, opsga, ar, inv, ap):
#     N = len(sls)+1 #N+1
#     yr, df = ['Year '+str(i) for i in np.arange(0,N)], pd.DataFrame()
#     for i in np.arange(0,N):
#         net_wc = delta_working_capital(net_AR=ar[i], 
#                                        net_inventory=inv[i], 
#                                        net_AP=ap[i])
#         inc_stm = income_statement(name=yr[i], 
#                                    sales=sls[i], 
#                                    cogs=cogs[i], 
#                                    sga=opsga[i], 
#                                    da=0, interest=0, tax_rate=.4)
#         inc_stm = inc_stm.get_income_statement(depreciation_rate=0, 
#                                      delta_capex=0, 
#                                      delta_workingcap=net_wc).transpose()
#         df = df.append(inc_stm)

#     df = df.transpose()

#     PERP_CF = df.ix['FreeCashFlow'][-1:]
#     df.ix['FreeCashFlow'][-2] = df.ix['FreeCashFlow'][-2] + perpetuity(PERP_CF, r=discount_rate)
#     cashflows = df.ix['FreeCashFlow'][:-1].tolist()


#     print('''
#     Adjusted DF with second to last period adjusted for perpetuity.
#     ''')
#     df.drop(labels=df.columns[-1], axis=1, inplace=True)
#     print(df)


#     NPV = np.npv(discount_rate, cashflows)

#     print(PBP, NPV, IRR)

#     print('''
#     NPV w/ Perpetuity ................................ %.4f
#     '''%(NPV))


           Sales
Year            
2017      0.0000
2018   1050.0000
2019   1102.5000
2020   1157.6300
2021  25754.9489


21767.05858752429